## **MYMENSINGH**

In [ ]:
# ===============================
#  Bangla Dialect → Standard Bangla using mBART-50 (Improved)
# ===============================

!pip install transformers[sentencepiece] datasets sacrebleu evaluate torch pandas openpyxl --quiet

import pandas as pd
import numpy as np
import evaluate
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
)
import torch
import gc # <-- Added for memory cleanup
from google.colab import drive

print("✅ Libraries installed and imported successfully.")
#drive.mount('/content/drive')

# ===============================
#  Load Dataset
# ===============================

try:
    df = pd.read_excel("/content/bangla_dialect_aligned_18920.xlsx")
    print("✅ Successfully loaded the dataset")
except FileNotFoundError:
    print("⚠️ Dataset not found — using sample data.")
    df = pd.DataFrame({
        'Standard_Bangla': ["সে স্কুলে যায়。"],
        'Barisal': ["হেই ইস্কুলে যায়。"],
        'Chittagong': ["হেই স্কোলে যায়。"],
        'Sylhet': ["তারে ইস্কুলে যায়。"]
    })

# Choose dialects to train
DIALECTS_TO_TRAIN = ['Barisal'] # Can now train multiple, one after another

# ===============================
#  Load Tokenizer (mBART)
# ===============================

MODEL_NAME = "facebook/mbart-large-50-many-to-many-mmt"

# Load tokenizer once, it can be reused
# For mBART, we set the src/tgt lang on the tokenizer instance
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.src_lang = "bn_IN"
tokenizer.tgt_lang = "bn_IN"

print(f"\n✅ Tokenizer for '{MODEL_NAME}' loaded.")
print(f" Language pair: bn_IN → bn_IN (Dialect → Standard).")


# ===============================
#  Helper Functions
# ===============================

def create_dataset_dict(dialect_col):
    """Creates a preprocessed DatasetDict for a given dialect."""
    print(f"\n--- Processing dialect: {dialect_col} ---")
    df_clean = df[['Standard_Bangla', dialect_col]].dropna()
    df_clean = df_clean[
        (df_clean['Standard_Bangla'].apply(lambda x: isinstance(x, str) and x.strip() != "")) &
        (df_clean[dialect_col].apply(lambda x: isinstance(x, str) and x.strip() != ""))
    ]
    subset_df = df_clean.rename(columns={'Standard_Bangla': 'target', dialect_col: 'source'})

    if len(subset_df) < 20: # Need enough data to split
        print(f"⚠️ Insufficient data for {dialect_col}. Skipping.")
        return None

    hf_dataset = Dataset.from_pandas(subset_df)

    # Split, ensuring splits are not too small
    train_test_split = hf_dataset.train_test_split(test_size=0.2, seed=42)
    test_val_split = train_test_split['test'].train_test_split(test_size=0.5, seed=42)

    dataset_dict = DatasetDict({
        'train': train_test_split['train'],
        'validation': test_val_split['train'],
        'test': test_val_split['test']
    })
    print(f"✅ Dataset splits created for {dialect_col}: Train {len(dataset_dict['train'])}, Val {len(dataset_dict['validation'])}, Test {len(dataset_dict['test'])}")
    return dataset_dict


def tokenize_and_prepare_datasets(dataset_dict):
    """Tokenizes the source and target text in the dataset."""

    def tokenize_fn(examples):
        # mBART tokenizer uses the `src_lang` set on the tokenizer
        model_inputs = tokenizer(
            examples["source"],
            max_length=128,
            truncation=True,
            padding="max_length"
        )
        # mBART tokenizer uses `tgt_lang` when `text_target` is provided
        labels = tokenizer(
            text_target=examples["target"],
            max_length=128,
            truncation=True,
            padding="max_length"
        )
        model_inputs["labels"] = labels["input_ids"]
        return model_inputs

    tokenized_datasets = dataset_dict.map(
        tokenize_fn,
        batched=True,
        remove_columns=['source', 'target'] # <-- Added remove_columns
    )
    print("✅ Tokenization complete.")
    return tokenized_datasets


def train_and_evaluate(dialect_name, train_ds, val_ds, test_ds):
    """Initializes and runs the training, then evaluates on the test set."""

    # --- Memory Cleanup ---
    gc.collect()
    torch.cuda.empty_cache()
    # ----------------------

    output_dir = f"/content/Bangla_Dialect_Models/mbart-bangla-{dialect_name.lower()}"

    # --- Load fresh model ---
    model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)
    # ------------------------

    data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
    bleu_metric = evaluate.load("sacrebleu")

    def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
        decoded_preds = [pred.strip() for pred in decoded_preds]
        decoded_labels = [[label.strip()] for label in decoded_labels]
        result = bleu_metric.compute(predictions=decoded_preds, references=decoded_labels)
        return {"bleu": result["score"]}

    # --- Improved Training Arguments ---
    training_args = Seq2SeqTrainingArguments(
        output_dir=output_dir,
        learning_rate=2e-5,
        per_device_train_batch_size=4,
        gradient_accumulation_steps=2, # <-- Effective batch size = 8

        per_device_eval_batch_size=8,  # <-- Can be larger for eval
        weight_decay=0.01,
        save_total_limit=2,

        # Increased epochs to avoid underfitting
        num_train_epochs=5,           # <-- Set to 5

        predict_with_generate=True,
        fp16=torch.cuda.is_available(),

        # --- CRITICAL FIX: To save the best model ---
        load_best_model_at_end=True,
        metric_for_best_model="bleu",
        # --------------------------------------------

        # Your "step" settings
        eval_strategy="steps",
        eval_steps=500,
        save_steps=500,     # Must match eval_steps
        logging_steps=100,

        # Standard good practices
        warmup_steps=300,
        max_grad_norm=1.0,
        generation_max_length=128,
        generation_num_beams=4,

        # Added to hide wandb logs
        report_to="none",
    )
    # -----------------------------------

    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    print(f"\n Starting training for {dialect_name} dialect...")
    trainer.train()

    print(f"\n Evaluating on the test set for {dialect_name}...")
    # --- Improved: Use original dataset for printing ---
    original_test_ds = split_dataset_dict["test"]

    test_results = trainer.predict(test_ds)
    final_bleu_score = test_results.metrics.get('test_bleu', 0.0)
    print(f" Test Set BLEU Score for {dialect_name}: {final_bleu_score:.2f}")

    print("\n🔍 Example Translations:")
    predictions = tokenizer.batch_decode(test_results.predictions, skip_special_tokens=True)
    for i in range(min(5, len(predictions))):
        print(f" Input (Dialect):   {original_test_ds[i]['source']}")
        print(f" Actual (Standard): {original_test_ds[i]['target']}")
        print(f" Predicted:         {predictions[i]}\n")
    # -------------------------------------------------

    # Save the final (best) model
    trainer.save_model(f"{output_dir}/best_model")
    print(f"✅ Best model for {dialect_name} saved to {output_dir}/best_model")

    return trainer


# ===============================
# 🚀 Training Loop (Improved)
# ===============================
for dialect in DIALECTS_TO_TRAIN:
    split_dataset_dict = create_dataset_dict(dialect)

    if split_dataset_dict is None:
        continue # Skip if dataset creation failed

    tokenized_datasets = tokenize_and_prepare_datasets(split_dataset_dict)

    # The train_and_evaluate function now handles model loading and saving
    trained_trainer = train_and_evaluate(
        dialect,
        tokenized_datasets["train"],
        tokenized_datasets["validation"],
        tokenized_datasets["test"]
    )

    print(f"--- Finished processing {dialect} ---")


print("\n✅ Finished training all dialect models. Best models are saved in Google Drive.")

✅ Libraries installed and imported successfully.
✅ Successfully loaded the dataset

✅ Tokenizer for 'facebook/mbart-large-50-many-to-many-mmt' loaded.
 Language pair: bn_IN → bn_IN (Dialect → Standard).

--- Processing dialect: Barisal ---
✅ Dataset splits created for Barisal: Train 2784, Val 348, Test 348


Map:   0%|          | 0/2784 [00:00<?, ? examples/s]

Map:   0%|          | 0/348 [00:00<?, ? examples/s]

Map:   0%|          | 0/348 [00:00<?, ? examples/s]

✅ Tokenization complete.


...


 Starting training for Barisal dialect...


Step,Training Loss,Validation Loss,Bleu
500,0.125700,0.113813,35.764739
1000,0.060400,0.089560,44.350685
1500,0.030300,0.088473,50.370624


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].



 Evaluating on the test set for Barisal...


 Test Set BLEU Score for Barisal: 47.85

🔍 Example Translations:
 Input (Dialect):   আম্নে কি মোর প্রশ্নের জবাব দেতে পারেন?
 Actual (Standard): আপনি কি আমার প্রশ্নের জবাব দিতে পারেন?
 Predicted:         আপনি কি আমার পছন্দের জবাব দিতে পারেন?

 Input (Dialect):   আকাশের নীল রংডা অসাধারন
 Actual (Standard): আকাশের নীল রঙটি অসাধারণ
 Predicted:         আকাশের নীল রংটা অসাধারণ

 Input (Dialect):   ছোডো বুইন চিল্লান দিয়া ওঠছে দারুণ
 Actual (Standard): ছোটো বোন চিৎকার দিয়ে উঠল, দারুণ
 Predicted:         ছোট বোন চিৎ থেকে উঠেছে অসাধারণ

 Input (Dialect):   আমনে খেলায় জেতছেন
 Actual (Standard): আপনি খেলায় জিতলেন
 Predicted:         আপনি খেলায় জিতলেন

 Input (Dialect):   হারাইয়া গেছি
 Actual (Standard): হারিয়ে গেছি
 Predicted:         হারিয়ে গেছি

✅ Best model for Barisal saved to /content/Bangla_Dialect_Models/mbart-bangla-barisal/best_model
--- Finished processing Barisal ---

✅ Finished training all dialect models. Best models are saved in Google Drive.


## **BARISAL**

In [ ]:
# ===============================
#  Bangla Dialect → Standard Bangla using mBART-50 (Improved)
# ===============================

!pip install transformers[sentencepiece] datasets sacrebleu evaluate torch pandas openpyxl --quiet

import pandas as pd
import numpy as np
import evaluate
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
)
import torch
import gc # <-- Added for memory cleanup
from google.colab import drive

print("✅ Libraries installed and imported successfully.")
#drive.mount('/content/drive')

# ===============================
#  Load Dataset
# ===============================

try:
    df = pd.read_excel("/content/bangla_dialect_aligned_18920.xlsx")
    print("✅ Successfully loaded the dataset")
except FileNotFoundError:
    print("⚠️ Dataset not found — using sample data.")
    df = pd.DataFrame({
        'Standard_Bangla': ["সে স্কুলে যায়。"],
        'Barisal': ["হেই ইস্কুলে যায়。"],
        'Chittagong': ["হেই স্কোলে যায়。"],
        'Sylhet': ["তারে ইস্কুলে যায়。"]
    })

# Choose dialects to train
DIALECTS_TO_TRAIN = ['Barisal'] # Can now train multiple, one after another

# ===============================
#  Load Tokenizer (mBART)
# ===============================

MODEL_NAME = "facebook/mbart-large-50-many-to-many-mmt"

# Load tokenizer once, it can be reused
# For mBART, we set the src/tgt lang on the tokenizer instance
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.src_lang = "bn_IN"
tokenizer.tgt_lang = "bn_IN"

print(f"\n✅ Tokenizer for '{MODEL_NAME}' loaded.")
print(f" Language pair: bn_IN → bn_IN (Dialect → Standard).")


# ===============================
#  Helper Functions
# ===============================

def create_dataset_dict(dialect_col):
    """Creates a preprocessed DatasetDict for a given dialect."""
    print(f"\n--- Processing dialect: {dialect_col} ---")
    df_clean = df[['Standard_Bangla', dialect_col]].dropna()
    df_clean = df_clean[
        (df_clean['Standard_Bangla'].apply(lambda x: isinstance(x, str) and x.strip() != "")) &
        (df_clean[dialect_col].apply(lambda x: isinstance(x, str) and x.strip() != ""))
    ]
    subset_df = df_clean.rename(columns={'Standard_Bangla': 'target', dialect_col: 'source'})

    if len(subset_df) < 20: # Need enough data to split
        print(f"⚠️ Insufficient data for {dialect_col}. Skipping.")
        return None

    hf_dataset = Dataset.from_pandas(subset_df)

    # Split, ensuring splits are not too small
    train_test_split = hf_dataset.train_test_split(test_size=0.2, seed=42)
    test_val_split = train_test_split['test'].train_test_split(test_size=0.5, seed=42)

    dataset_dict = DatasetDict({
        'train': train_test_split['train'],
        'validation': test_val_split['train'],
        'test': test_val_split['test']
    })
    print(f"✅ Dataset splits created for {dialect_col}: Train {len(dataset_dict['train'])}, Val {len(dataset_dict['validation'])}, Test {len(dataset_dict['test'])}")
    return dataset_dict


def tokenize_and_prepare_datasets(dataset_dict):
    """Tokenizes the source and target text in the dataset."""

    def tokenize_fn(examples):
        # mBART tokenizer uses the `src_lang` set on the tokenizer
        model_inputs = tokenizer(
            examples["source"],
            max_length=128,
            truncation=True,
            padding="max_length"
        )
        # mBART tokenizer uses `tgt_lang` when `text_target` is provided
        labels = tokenizer(
            text_target=examples["target"],
            max_length=128,
            truncation=True,
            padding="max_length"
        )
        model_inputs["labels"] = labels["input_ids"]
        return model_inputs

    tokenized_datasets = dataset_dict.map(
        tokenize_fn,
        batched=True,
        remove_columns=['source', 'target'] # <-- Added remove_columns
    )
    print("✅ Tokenization complete.")
    return tokenized_datasets


def train_and_evaluate(dialect_name, train_ds, val_ds, test_ds):
    """Initializes and runs the training, then evaluates on the test set."""

    # --- Memory Cleanup ---
    gc.collect()
    torch.cuda.empty_cache()
    # ----------------------

    output_dir = f"/content/Bangla_Dialect_Models/mbart-bangla-{dialect_name.lower()}"

    # --- Load fresh model ---
    model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)
    # ------------------------

    data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
    bleu_metric = evaluate.load("sacrebleu")

    def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
        decoded_preds = [pred.strip() for pred in decoded_preds]
        decoded_labels = [[label.strip()] for label in decoded_labels]
        result = bleu_metric.compute(predictions=decoded_preds, references=decoded_labels)
        return {"bleu": result["score"]}

    # --- Improved Training Arguments ---
    training_args = Seq2SeqTrainingArguments(
        output_dir=output_dir,
        learning_rate=2e-5,

        # Use gradient accumulation for a larger effective batch size
        per_device_train_batch_size=4,
        gradient_accumulation_steps=2, # <-- Effective batch size = 8

        per_device_eval_batch_size=8,  # <-- Can be larger for eval
        weight_decay=0.01,
        save_total_limit=2,

        # Increased epochs to avoid underfitting
        num_train_epochs=5,           # <-- Set to 5

        predict_with_generate=True,
        fp16=torch.cuda.is_available(),

        # --- CRITICAL FIX: To save the best model ---
        load_best_model_at_end=True,
        metric_for_best_model="bleu",
        # --------------------------------------------

        # Your "step" settings
        eval_strategy="steps",
        eval_steps=500,
        save_steps=500,     # Must match eval_steps
        logging_steps=100,

        # Standard good practices
        warmup_steps=300,
        max_grad_norm=1.0,
        generation_max_length=128,
        generation_num_beams=4,

        # Added to hide wandb logs
        report_to="none",
    )
    # -----------------------------------

    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    print(f"\n Starting training for {dialect_name} dialect...")
    trainer.train()

    print(f"\n Evaluating on the test set for {dialect_name}...")
    # --- Improved: Use original dataset for printing ---
    original_test_ds = split_dataset_dict["test"]

    test_results = trainer.predict(test_ds)
    final_bleu_score = test_results.metrics.get('test_bleu', 0.0)
    print(f" Test Set BLEU Score for {dialect_name}: {final_bleu_score:.2f}")

    print("\n🔍 Example Translations:")
    predictions = tokenizer.batch_decode(test_results.predictions, skip_special_tokens=True)
    for i in range(min(5, len(predictions))):
        print(f" Input (Dialect):   {original_test_ds[i]['source']}")
        print(f" Actual (Standard): {original_test_ds[i]['target']}")
        print(f" Predicted:         {predictions[i]}\n")
    # -------------------------------------------------

    # Save the final (best) model
    trainer.save_model(f"{output_dir}/best_model")
    print(f"✅ Best model for {dialect_name} saved to {output_dir}/best_model")

    return trainer


# ===============================
# 🚀 Training Loop (Improved)
# ===============================
for dialect in DIALECTS_TO_TRAIN:
    split_dataset_dict = create_dataset_dict(dialect)

    if split_dataset_dict is None:
        continue # Skip if dataset creation failed

    tokenized_datasets = tokenize_and_prepare_datasets(split_dataset_dict)

    # The train_and_evaluate function now handles model loading and saving
    trained_trainer = train_and_evaluate(
        dialect,
        tokenized_datasets["train"],
        tokenized_datasets["validation"],
        tokenized_datasets["test"]
    )

    print(f"--- Finished processing {dialect} ---")


print("\n✅ Finished training all dialect models. Best models are saved in Google Drive.")

✅ Libraries installed and imported successfully.
✅ Successfully loaded the dataset

✅ Tokenizer for 'facebook/mbart-large-50-many-to-many-mmt' loaded.
 Language pair: bn_IN → bn_IN (Dialect → Standard).

--- Processing dialect: Barisal ---
✅ Dataset splits created for Barisal: Train 2784, Val 348, Test 348


Map:   0%|          | 0/2784 [00:00<?, ? examples/s]

Map:   0%|          | 0/348 [00:00<?, ? examples/s]

Map:   0%|          | 0/348 [00:00<?, ? examples/s]

✅ Tokenization complete.


...


 Starting training for Barisal dialect...


Step,Training Loss,Validation Loss,Bleu
500,0.125700,0.113813,35.764739
1000,0.060400,0.089560,44.350685
1500,0.030300,0.088473,50.370624


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].



 Evaluating on the test set for Barisal...


 Test Set BLEU Score for Barisal: 47.85

🔍 Example Translations:
 Input (Dialect):   আম্নে কি মোর প্রশ্নের জবাব দেতে পারেন?
 Actual (Standard): আপনি কি আমার প্রশ্নের জবাব দিতে পারেন?
 Predicted:         আপনি কি আমার পছন্দের জবাব দিতে পারেন?

 Input (Dialect):   আকাশের নীল রংডা অসাধারন
 Actual (Standard): আকাশের নীল রঙটি অসাধারণ
 Predicted:         আকাশের নীল রংটা অসাধারণ

 Input (Dialect):   ছোডো বুইন চিল্লান দিয়া ওঠছে দারুণ
 Actual (Standard): ছোটো বোন চিৎকার দিয়ে উঠল, দারুণ
 Predicted:         ছোট বোন চিৎ থেকে উঠেছে অসাধারণ

 Input (Dialect):   আমনে খেলায় জেতছেন
 Actual (Standard): আপনি খেলায় জিতলেন
 Predicted:         আপনি খেলায় জিতলেন

 Input (Dialect):   হারাইয়া গেছি
 Actual (Standard): হারিয়ে গেছি
 Predicted:         হারিয়ে গেছি

✅ Best model for Barisal saved to /content/Bangla_Dialect_Models/mbart-bangla-barisal/best_model
--- Finished processing Barisal ---

✅ Finished training all dialect models. Best models are saved in Google Drive.


## **SYLHET**


In [ ]:
# ===============================
#  Bangla Dialect → Standard Bangla using mBART-50 (Improved)
# ===============================

!pip install transformers[sentencepiece] datasets sacrebleu evaluate torch pandas openpyxl --quiet

import pandas as pd
import numpy as np
import evaluate
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
)
import torch
import gc # <-- Added for memory cleanup
from google.colab import drive

print("✅ Libraries installed and imported successfully.")
#drive.mount('/content/drive')

# ===============================
#  Load Dataset
# ===============================

try:
    df = pd.read_excel("/content/bangla_dialect_aligned_18920.xlsx")
    print("✅ Successfully loaded the dataset")
except FileNotFoundError:
    print("⚠️ Dataset not found — using sample data.")
    df = pd.DataFrame({
        'Standard_Bangla': ["সে স্কুলে যায়。"],
        'Barisal': ["হেই ইস্কুলে যায়。"],
        'Chittagong': ["হেই স্কোলে যায়。"],
        'Sylhet': ["তারে ইস্কুলে যায়。"]
    })

# Choose dialects to train
DIALECTS_TO_TRAIN = ['Sylhet'] # Can now train multiple, one after another

# ===============================
#  Load Tokenizer (mBART)
# ===============================

MODEL_NAME = "facebook/mbart-large-50-many-to-many-mmt"

# Load tokenizer once, it can be reused
# For mBART, we set the src/tgt lang on the tokenizer instance
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.src_lang = "bn_IN"
tokenizer.tgt_lang = "bn_IN"

print(f"\n✅ Tokenizer for '{MODEL_NAME}' loaded.")
print(f" Language pair: bn_IN → bn_IN (Dialect → Standard).")


# ===============================
#  Helper Functions
# ===============================

def create_dataset_dict(dialect_col):
    """Creates a preprocessed DatasetDict for a given dialect."""
    print(f"\n--- Processing dialect: {dialect_col} ---")
    df_clean = df[['Standard_Bangla', dialect_col]].dropna()
    df_clean = df_clean[
        (df_clean['Standard_Bangla'].apply(lambda x: isinstance(x, str) and x.strip() != "")) &
        (df_clean[dialect_col].apply(lambda x: isinstance(x, str) and x.strip() != ""))
    ]
    subset_df = df_clean.rename(columns={'Standard_Bangla': 'target', dialect_col: 'source'})

    if len(subset_df) < 20: # Need enough data to split
        print(f"⚠️ Insufficient data for {dialect_col}. Skipping.")
        return None

    hf_dataset = Dataset.from_pandas(subset_df)

    # Split, ensuring splits are not too small
    train_test_split = hf_dataset.train_test_split(test_size=0.2, seed=42)
    test_val_split = train_test_split['test'].train_test_split(test_size=0.5, seed=42)

    dataset_dict = DatasetDict({
        'train': train_test_split['train'],
        'validation': test_val_split['train'],
        'test': test_val_split['test']
    })
    print(f"✅ Dataset splits created for {dialect_col}: Train {len(dataset_dict['train'])}, Val {len(dataset_dict['validation'])}, Test {len(dataset_dict['test'])}")
    return dataset_dict


def tokenize_and_prepare_datasets(dataset_dict):
    """Tokenizes the source and target text in the dataset."""

    def tokenize_fn(examples):
        # mBART tokenizer uses the `src_lang` set on the tokenizer
        model_inputs = tokenizer(
            examples["source"],
            max_length=128,
            truncation=True,
            padding="max_length"
        )
        # mBART tokenizer uses `tgt_lang` when `text_target` is provided
        labels = tokenizer(
            text_target=examples["target"],
            max_length=128,
            truncation=True,
            padding="max_length"
        )
        model_inputs["labels"] = labels["input_ids"]
        return model_inputs

    tokenized_datasets = dataset_dict.map(
        tokenize_fn,
        batched=True,
        remove_columns=['source', 'target'] # <-- Added remove_columns
    )
    print("✅ Tokenization complete.") # Corrected indentation
    return tokenized_datasets


def train_and_evaluate(dialect_name, train_ds, val_ds, test_ds):
    """Initializes and runs the training, then evaluates on the test set."""

    # --- Memory Cleanup ---
    gc.collect()
    torch.cuda.empty_cache()
    # ----------------------

    output_dir = f"/content/Bangla_Dialect_Models/mbart-bangla-{dialect_name.lower()}"

    # --- Load fresh model ---
    model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)
    # ------------------------

    data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
    bleu_metric = evaluate.load("sacrebleu")

    def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
        decoded_preds = [pred.strip() for pred in decoded_preds]
        decoded_labels = [[label.strip()] for label in decoded_labels]
        result = bleu_metric.compute(predictions=decoded_preds, references=decoded_labels)
        return {"bleu": result["score"]}

    # --- Improved Training Arguments ---
    training_args = Seq2SeqTrainingArguments(
        output_dir=output_dir,
        learning_rate=2e-5,

        # Use gradient accumulation for a larger effective batch size
        per_device_train_batch_size=4,
        gradient_accumulation_steps=2, # <-- Effective batch size = 8

        per_device_eval_batch_size=8,  # <-- Can be larger for eval
        weight_decay=0.01,
        save_total_limit=2,

        # Increased epochs to avoid underfitting
        num_train_epochs=5,           # <-- Set to 5

        predict_with_generate=True,
        fp16=torch.cuda.is_available(),

        # --- CRITICAL FIX: To save the best model ---
        load_best_model_at_end=True,
        metric_for_best_model="bleu",
        # --------------------------------------------

        # Your "step" settings
        eval_strategy="steps",
        eval_steps=500,
        save_steps=500,     # Must match eval_steps
        logging_steps=100,

        # Standard good practices
        warmup_steps=300,
        max_grad_norm=1.0,
        generation_max_length=128,
        generation_num_beams=4,

        # Added to hide wandb logs
        report_to="none",
    )
    # -----------------------------------

    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    print(f"\n Starting training for {dialect_name} dialect...")
    trainer.train()

    print(f"\n Evaluating on the test set for {dialect_name}...")
    # --- Improved: Use original dataset for printing ---
    original_test_ds = split_dataset_dict["test"]

    test_results = trainer.predict(test_ds)
    final_bleu_score = test_results.metrics.get('test_bleu', 0.0)
    print(f" Test Set BLEU Score for {dialect_name}: {final_bleu_score:.2f}")

    print("\n🔍 Example Translations:")
    predictions = tokenizer.batch_decode(test_results.predictions, skip_special_tokens=True)
    for i in range(min(5, len(predictions))):
        print(f" Input (Dialect):   {original_test_ds[i]['source']}")
        print(f" Actual (Standard): {original_test_ds[i]['target']}")
        print(f" Predicted:         {predictions[i]}\n")
    # -------------------------------------------------

    # Save the final (best) model
    trainer.save_model(f"{output_dir}/best_model")
    print(f"✅ Best model for {dialect_name} saved to {output_dir}/best_model")

    return trainer


# ===============================
# 🚀 Training Loop (Improved)
# ===============================
for dialect in DIALECTS_TO_TRAIN:
    split_dataset_dict = create_dataset_dict(dialect)

    if split_dataset_dict is None:
        continue # Skip if dataset creation failed

    tokenized_datasets = tokenize_and_prepare_datasets(split_dataset_dict)

    # The train_and_evaluate function now handles model loading and saving
    trained_trainer = train_and_evaluate(
        dialect,
        tokenized_datasets["train"],
        tokenized_datasets["validation"],
        tokenized_datasets["test"]
    )

    print(f"--- Finished processing {dialect} ---")


print("\n✅ Finished training all dialect models. Best models are saved in Google Drive.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.5 MB/s eta 0:00:00
✅ Libraries installed and imported successfully.
✅ Successfully loaded the dataset


tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

...


 Starting training for Sylhet dialect...


Step,Training Loss,Validation Loss,Bleu
500,0.138400,0.131626,28.566290
1000,0.072100,0.108993,37.558164
1500,0.035700,0.108468,41.329051


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].



 Evaluating on the test set for Sylhet...


 Test Set BLEU Score for Sylhet: 38.55

🔍 Example Translations:
 Input (Dialect):   আফনে কিতা আমার প্রশ্ন'র জবাব দিতে ফারবা নি?
 Actual (Standard): আপনি কি আমার প্রশ্নের জবাব দিতে পারেন?
 Predicted:         আপনি কি আমার বন্ধুর জবাব দিতে পারবেন?

 Input (Dialect):   আকাশ ওর নীল রঙটা অসাধারণ
 Actual (Standard): আকাশের নীল রঙটি অসাধারণ
 Predicted:         আকাশের নীল রঙটা অসাধারণ

 Input (Dialect):   ছোট বইনে চিল্লাইয়া উঠলো, দারুণ
 Actual (Standard): ছোটো বোন চিৎকার দিয়ে উঠল, দারুণ
 Predicted:         ছোটো বোন চিরিয়ে উঠলো,দারুন

 Input (Dialect):   আফনে খেলাত জিতলা
 Actual (Standard): আপনি খেলায় জিতলেন
 Predicted:         আপনি খেলায় জিতলেন

 Input (Dialect):   আমি হারি গেছি
 Actual (Standard): হারিয়ে গেছি
 Predicted:         আমি হারিয়ে গেছি



## **CHITTAGONG**

In [ ]:
# ===============================
#  Bangla Dialect → Standard Bangla using mBART-50 (Improved)
# ===============================

!pip install transformers[sentencepiece] datasets sacrebleu evaluate torch pandas openpyxl --quiet

import pandas as pd
import numpy as np
import evaluate
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
)
import torch
import gc # <-- Added for memory cleanup
from google.colab import drive

print("✅ Libraries installed and imported successfully.")
#drive.mount('/content/drive')

# ===============================
#  Load Dataset
# ===============================

try:
    df = pd.read_excel("/content/bangla_dialect_aligned_18920.xlsx")
    print("✅ Successfully loaded the dataset")
except FileNotFoundError:
    print("⚠️ Dataset not found — using sample data.")
    df = pd.DataFrame({
        'Standard_Bangla': ["সে স্কুলে যায়。"],
        'Barisal': ["হেই ইস্কুলে যায়。"],
        'Chittagong': ["হেই স্কোলে যায়。"],
        'Sylhet': ["তারে ইস্কুলে যায়。"]
    })

# Choose dialects to train
DIALECTS_TO_TRAIN = ['Chittagong'] # Can now train multiple, one after another

# ===============================
#  Load Tokenizer (mBART)
# ===============================

MODEL_NAME = "facebook/mbart-large-50-many-to-many-mmt"

# Load tokenizer once, it can be reused
# For mBART, we set the src/tgt lang on the tokenizer instance
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.src_lang = "bn_IN"
tokenizer.tgt_lang = "bn_IN"

print(f"\n✅ Tokenizer for '{MODEL_NAME}' loaded.")
print(f" Language pair: bn_IN → bn_IN (Dialect → Standard).")


# ===============================
#  Helper Functions
# ===============================

def create_dataset_dict(dialect_col):
    """Creates a preprocessed DatasetDict for a given dialect."""
    print(f"\n--- Processing dialect: {dialect_col} ---")
    df_clean = df[['Standard_Bangla', dialect_col]].dropna()
    df_clean = df_clean[
        (df_clean['Standard_Bangla'].apply(lambda x: isinstance(x, str) and x.strip() != "")) &
        (df_clean[dialect_col].apply(lambda x: isinstance(x, str) and x.strip() != ""))
    ]
    subset_df = df_clean.rename(columns={'Standard_Bangla': 'target', dialect_col: 'source'})

    if len(subset_df) < 20: # Need enough data to split
        print(f"⚠️ Insufficient data for {dialect_col}. Skipping.")
        return None

    hf_dataset = Dataset.from_pandas(subset_df)

    # Split, ensuring splits are not too small
    train_test_split = hf_dataset.train_test_split(test_size=0.2, seed=42)
    test_val_split = train_test_split['test'].train_test_split(test_size=0.5, seed=42)

    dataset_dict = DatasetDict({
        'train': train_test_split['train'],
        'validation': test_val_split['train'],
        'test': test_val_split['test']
    })
    print(f"✅ Dataset splits created for {dialect_col}: Train {len(dataset_dict['train'])}, Val {len(dataset_dict['validation'])}, Test {len(dataset_dict['test'])}")
    return dataset_dict


def tokenize_and_prepare_datasets(dataset_dict):
    """Tokenizes the source and target text in the dataset."""

    def tokenize_fn(examples):
        # mBART tokenizer uses the `src_lang` set on the tokenizer
        model_inputs = tokenizer(
            examples["source"],
            max_length=128,
            truncation=True,
            padding="max_length"
        )
        # mBART tokenizer uses `tgt_lang` when `text_target` is provided
        labels = tokenizer(
            text_target=examples["target"],
            max_length=128,
            truncation=True,
            padding="max_length"
        )
        model_inputs["labels"] = labels["input_ids"]
        return model_inputs

    tokenized_datasets = dataset_dict.map(
        tokenize_fn,
        batched=True,
        remove_columns=['source', 'target'] # <-- Added remove_columns
    )
    print("✅ Tokenization complete.") # Corrected indentation
    return tokenized_datasets


def train_and_evaluate(dialect_name, train_ds, val_ds, test_ds):
    """Initializes and runs the training, then evaluates on the test set."""

    # --- Memory Cleanup ---
    gc.collect()
    torch.cuda.empty_cache()
    # ----------------------

    output_dir = f"/content/Bangla_Dialect_Models/mbart-bangla-{dialect_name.lower()}"

    # --- Load fresh model ---
    model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)
    # ------------------------

    data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
    bleu_metric = evaluate.load("sacrebleu")

    def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
        decoded_preds = [pred.strip() for pred in decoded_preds]
        decoded_labels = [[label.strip()] for label in decoded_labels]
        result = bleu_metric.compute(predictions=decoded_preds, references=decoded_labels)
        return {"bleu": result["score"]}

    # --- Improved Training Arguments ---
    training_args = Seq2SeqTrainingArguments(
        output_dir=output_dir,
        learning_rate=2e-5,

        # Use gradient accumulation for a larger effective batch size
        per_device_train_batch_size=4,
        gradient_accumulation_steps=2, # <-- Effective batch size = 8

        per_device_eval_batch_size=8,  # <-- Can be larger for eval
        weight_decay=0.01,
        save_total_limit=2,

        # Increased epochs to avoid underfitting
        num_train_epochs=5,           # <-- Set to 5

        predict_with_generate=True,
        fp16=torch.cuda.is_available(),

        # --- CRITICAL FIX: To save the best model ---
        load_best_model_at_end=True,
        metric_for_best_model="bleu",
        # --------------------------------------------

        # Your "step" settings
        eval_strategy="steps",
        eval_steps=500,
        save_steps=500,     # Must match eval_steps
        logging_steps=100,

        # Standard good practices
        warmup_steps=300,
        max_grad_norm=1.0,
        generation_max_length=128,
        generation_num_beams=4,

        # Added to hide wandb logs
        report_to="none",
    )
    # -----------------------------------

    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    print(f"\n Starting training for {dialect_name} dialect...")
    trainer.train()

    print(f"\n Evaluating on the test set for {dialect_name}...")
    # --- Improved: Use original dataset for printing ---
    original_test_ds = split_dataset_dict["test"]

    test_results = trainer.predict(test_ds)
    final_bleu_score = test_results.metrics.get('test_bleu', 0.0)
    print(f" Test Set BLEU Score for {dialect_name}: {final_bleu_score:.2f}")

    print("\n🔍 Example Translations:")
    predictions = tokenizer.batch_decode(test_results.predictions, skip_special_tokens=True)
    for i in range(min(5, len(predictions))):
        print(f" Input (Dialect):   {original_test_ds[i]['source']}")
        print(f" Actual (Standard): {original_test_ds[i]['target']}")
        print(f" Predicted:         {predictions[i]}\n")
    # -------------------------------------------------

    # Save the final (best) model
    trainer.save_model(f"{output_dir}/best_model")
    print(f"✅ Best model for {dialect_name} saved to {output_dir}/best_model")

    return trainer


# ===============================
# 🚀 Training Loop (Improved)
# ===============================
for dialect in DIALECTS_TO_TRAIN:
    split_dataset_dict = create_dataset_dict(dialect)

    if split_dataset_dict is None:
        continue # Skip if dataset creation failed

    tokenized_datasets = tokenize_and_prepare_datasets(split_dataset_dict)

    # The train_and_evaluate function now handles model loading and saving
    trained_trainer = train_and_evaluate(
        dialect,
        tokenized_datasets["train"],
        tokenized_datasets["validation"],
        tokenized_datasets["test"]
    )

    print(f"--- Finished processing {dialect} ---")


print("\n✅ Finished training all dialect models. Best models are saved in Google Drive.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.0 MB/s eta 0:00:00
✅ Libraries installed and imported successfully.
✅ Successfully loaded the dataset


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

...


 Starting training for Chittagong dialect...


Step,Training Loss,Validation Loss,Bleu
500,0.172700,0.166116,20.532833
1000,0.090700,0.136891,32.607105
1500,0.046200,0.139361,33.216564


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].



 Evaluating on the test set for Chittagong...


 Test Set BLEU Score for Chittagong: 30.98

🔍 Example Translations:
 Input (Dialect):   অনে কি আঁর ফ্রশ্নর জবাব দিত ফারিবান নে
 Actual (Standard): আপনি কি আমার প্রশ্নের জবাব দিতে পারেন?
 Predicted:         আপনি কি আমার পছন্দের জবাব দিতে পারেন?

 Input (Dialect):   আস্সান অর নিল রং ইবা অসাদারন
 Actual (Standard): আকাশের নীল রঙটি অসাধারণ
 Predicted:         আকাশের নীল রঙটি অসাধারণ

 Input (Dialect):   ছোোড ভোইন চিৎকার দিয়েরে উডি গিয়ে
 Actual (Standard): ছোটো বোন চিৎকার দিয়ে উঠল, দারুণ
 Predicted:         ছোট বোন চিৎকার দিয়ে উঠে গিয়েছিল

 Input (Dialect):   ওনে হেলায় জিত্তন
 Actual (Standard): আপনি খেলায় জিতলেন
 Predicted:         আপনি হেরে উঠুন

 Input (Dialect):   হারী গিয়ি
 Actual (Standard): হারিয়ে গেছি
 Predicted:         হারিয়ে গেছি

✅ Best model for Chittagong saved to /content/Bangla_Dialect_Models/mbart-bangla-chittagong/best_model
--- Finished processing Chittagong ---

✅ Finished training all dialect models. Best models are saved in Google Drive.


## **NOAKHALI**

In [ ]:
# ===============================
#  Bangla Dialect → Standard Bangla using mBART-50 (Improved)
# ===============================

!pip install transformers[sentencepiece] datasets sacrebleu evaluate torch pandas openpyxl --quiet

import pandas as pd
import numpy as np
import evaluate
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
)
import torch
import gc # <-- Added for memory cleanup
from google.colab import drive

print("✅ Libraries installed and imported successfully.")
#drive.mount('/content/drive')

# ===============================
#  Load Dataset
# ===============================

try:
    df = pd.read_excel("/content/bangla_dialect_aligned_18920.xlsx")
    print("✅ Successfully loaded the dataset")
except FileNotFoundError:
    print("⚠️ Dataset not found — using sample data.")
    df = pd.DataFrame({
        'Standard_Bangla': ["সে স্কুলে যায়。"],
        'Barisal': ["হেই ইস্কুলে যায়。"],
        'Chittagong': ["হেই স্কোলে যায়。"],
        'Sylhet': ["তারে ইস্কুলে যায়。"]
    })

# Choose dialects to train
DIALECTS_TO_TRAIN = ['Noakhali'] # Can now train multiple, one after another

# ===============================
#  Load Tokenizer (mBART)
# ===============================

MODEL_NAME = "facebook/mbart-large-50-many-to-many-mmt"

# Load tokenizer once, it can be reused
# For mBART, we set the src/tgt lang on the tokenizer instance
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.src_lang = "bn_IN"
tokenizer.tgt_lang = "bn_IN"

print(f"\n✅ Tokenizer for '{MODEL_NAME}' loaded.")
print(f" Language pair: bn_IN → bn_IN (Dialect → Standard).")


# ===============================
#  Helper Functions
# ===============================

def create_dataset_dict(dialect_col):
    """Creates a preprocessed DatasetDict for a given dialect."""
    print(f"\n--- Processing dialect: {dialect_col} ---")
    df_clean = df[['Standard_Bangla', dialect_col]].dropna()
    df_clean = df_clean[
        (df_clean['Standard_Bangla'].apply(lambda x: isinstance(x, str) and x.strip() != "")) &
        (df_clean[dialect_col].apply(lambda x: isinstance(x, str) and x.strip() != ""))
    ]
    subset_df = df_clean.rename(columns={'Standard_Bangla': 'target', dialect_col: 'source'})

    if len(subset_df) < 20: # Need enough data to split
        print(f"⚠️ Insufficient data for {dialect_col}. Skipping.")
        return None

    hf_dataset = Dataset.from_pandas(subset_df)

    # Split, ensuring splits are not too small
    train_test_split = hf_dataset.train_test_split(test_size=0.2, seed=42)
    test_val_split = train_test_split['test'].train_test_split(test_size=0.5, seed=42)

    dataset_dict = DatasetDict({
        'train': train_test_split['train'],
        'validation': test_val_split['train'],
        'test': test_val_split['test']
    })
    print(f"✅ Dataset splits created for {dialect_col}: Train {len(dataset_dict['train'])}, Val {len(dataset_dict['validation'])}, Test {len(dataset_dict['test'])}")
    return dataset_dict


def tokenize_and_prepare_datasets(dataset_dict):
    """Tokenizes the source and target text in the dataset."""

    def tokenize_fn(examples):
        # mBART tokenizer uses the `src_lang` set on the tokenizer
        model_inputs = tokenizer(
            examples["source"],
            max_length=128,
            truncation=True,
            padding="max_length"
        )
        # mBART tokenizer uses `tgt_lang` when `text_target` is provided
        labels = tokenizer(
            text_target=examples["target"],
            max_length=128,
            truncation=True,
            padding="max_length"
        )
        model_inputs["labels"] = labels["input_ids"]
        return model_inputs

    tokenized_datasets = dataset_dict.map(
        tokenize_fn,
        batched=True,
        remove_columns=['source', 'target'] # <-- Added remove_columns
    )
    print("✅ Tokenization complete.")
    return tokenized_datasets


def train_and_evaluate(dialect_name, train_ds, val_ds, test_ds):
    """Initializes and runs the training, then evaluates on the test set."""

    # --- Memory Cleanup ---
    gc.collect()
    torch.cuda.empty_cache()
    # ----------------------

    output_dir = f"/content/Bangla_Dialect_Models/mbart-bangla-{dialect_name.lower()}"

    # --- Load fresh model ---
    model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)
    # ------------------------

    data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
    bleu_metric = evaluate.load("sacrebleu")

    def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
        decoded_preds = [pred.strip() for pred in decoded_preds]
        decoded_labels = [[label.strip()] for label in decoded_labels]
        result = bleu_metric.compute(predictions=decoded_preds, references=decoded_labels)
        return {"bleu": result["score"]}

    # --- Improved Training Arguments ---
    training_args = Seq2SeqTrainingArguments(
        output_dir=output_dir,
        learning_rate=2e-5,

        # Use gradient accumulation for a larger effective batch size
        per_device_train_batch_size=4,
        gradient_accumulation_steps=2, # <-- Effective batch size = 8

        per_device_eval_batch_size=8,  # <-- Can be larger for eval
        weight_decay=0.01,
        save_total_limit=2,

        # Increased epochs to avoid underfitting
        num_train_epochs=5,           # <-- Set to 5

        predict_with_generate=True,
        fp16=torch.cuda.is_available(),

        # --- CRITICAL FIX: To save the best model ---
        load_best_model_at_end=True,
        metric_for_best_model="bleu",
        # --------------------------------------------

        # Your "step" settings
        eval_strategy="steps",
        eval_steps=500,
        save_steps=500,     # Must match eval_steps
        logging_steps=100,

        # Standard good practices
        warmup_steps=300,
        max_grad_norm=1.0,
        generation_max_length=128,
        generation_num_beams=4,

        # Added to hide wandb logs
        report_to="none",
    )
    # -----------------------------------

    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    print(f"\n Starting training for {dialect_name} dialect...")
    trainer.train()

    print(f"\n Evaluating on the test set for {dialect_name}...")
    # --- Improved: Use original dataset for printing ---
    original_test_ds = split_dataset_dict["test"]

    test_results = trainer.predict(test_ds)
    final_bleu_score = test_results.metrics.get('test_bleu', 0.0)
    print(f" Test Set BLEU Score for {dialect_name}: {final_bleu_score:.2f}")

    print("\n🔍 Example Translations:")
    predictions = tokenizer.batch_decode(test_results.predictions, skip_special_tokens=True)
    for i in range(min(5, len(predictions))):
        print(f" Input (Dialect):   {original_test_ds[i]['source']}")
        print(f" Actual (Standard): {original_test_ds[i]['target']}")
        print(f" Predicted:         {predictions[i]}\n")
    # -------------------------------------------------

    # Save the final (best) model
    trainer.save_model(f"{output_dir}/best_model")
    print(f"✅ Best model for {dialect_name} saved to {output_dir}/best_model")

    return trainer


# ===============================
# 🚀 Training Loop (Improved)
# ===============================
for dialect in DIALECTS_TO_TRAIN:
    split_dataset_dict = create_dataset_dict(dialect)

    if split_dataset_dict is None:
        continue # Skip if dataset creation failed

    tokenized_datasets = tokenize_and_prepare_datasets(split_dataset_dict)

    # The train_and_evaluate function now handles model loading and saving
    trained_trainer = train_and_evaluate(
        dialect,
        tokenized_datasets["train"],
        tokenized_datasets["validation"],
        tokenized_datasets["test"]
    )

    print(f"--- Finished processing {dialect} ---")


print("\n✅ Finished training all dialect models. Best models are saved in Google Drive.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.1 MB/s eta 0:00:00
✅ Libraries installed and imported successfully.
✅ Successfully loaded the dataset


tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

...


 Starting training for Noakhali dialect...


Step,Training Loss,Validation Loss,Bleu
500,0.162300,0.151910,28.633643
1000,0.059100,0.140994,35.489501


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].



 Evaluating on the test set for Noakhali...


 Test Set BLEU Score for Noakhali: 39.56

🔍 Example Translations:
 Input (Dialect):   যাওয়ার লগে লগে কী এককান শব্দ কইরলো
 Actual (Standard): যাওয়ার সাথে সাথে কী একটা শব্দ করল
 Predicted:         যাওয়ার সাথে সাথে কী একটা সুন্দর করলো

 Input (Dialect):   তুই কি আরে এই কাম আন করি দিতা হাইরবা নি?
 Actual (Standard): তুমি কি আমাকে এই কাজটি করে দিতে পারবে?
 Predicted:         তুমি কি আমাকে এই কাজটি দিয়ে দিতে পারবে ?

 Input (Dialect):   তোর হরিক্ষা কবে?
 Actual (Standard): তোর পরিক্ষা কবে ?
 Predicted:         তোমার পছন্দ কবে?

 Input (Dialect):   হেতে মেলা দিন আগে স্কুলের অগগা বইতালিরে ভালোবাইসতো
 Actual (Standard): সে অনেক দিন আগে তার বিদ্যালয়ের একটি মেয়েকে ভালোবাসতো
 Predicted:         সে অনেক দিন আগে স্কুলের একটি বোতালিকে ভালোবাসতো

 Input (Dialect):   কোনো ভাবেই ওয়াশা হানি দেয়র না
 Actual (Standard): কোনোভাবেই ওয়াসা পানি দিচ্ছে না
 Predicted:         কোনো ভাবেই আমশা পানি দেয় না

✅ Best model for Noakhali saved to /content/Bangla_Dialect_Models/mbart-bangla-noakhali/best_model
--- F